<a href="https://colab.research.google.com/github/makhijakabir/eyeDiseaseDetection/blob/main/eyeDiseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Eye Disease Detection system using CNN approach 


##Model Imports

In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [9]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from tqdm import tqdm

%matplotlib inline

In [6]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [8]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

##Model Concepts

This notebook implements a custom callback to adjust the learning rate during training.
The callback has a parameter dwell. If dwell is set to True, the callback monitors the
validation loss. It keeps track of the lowest validation loss thus far achieved as you run
through each epoch and stores this as the lowest loss and also stores the weights for that
epoch as the best weights. At the end of an epoch the validation loss for that epoch is
compared with the lowest loss. If the validation loss at the end of the current epoch is
less than the lowest loss than it becomes the lowest loss and the weights of the current
epoch become the best weights

If the validation loss at the end of the current epoch is greator than the lowest loss
this implies you have moved to a location in Nspace(N is the number of trainable parameters
on the validation cost function surface that is less favorable(higher cost) than the position
in Nspace defined by the best weights. Therefore why move the models weights to this less
favorable location? Better to reset the models weights to the best weights, then lower the
learning rate and run more epochs. The new learning rate is set to new_lr=current_lr * factor
where factor is a user specified parameter in the instantiation of the callback. By default
it is set to .04 and by default dwell is set to True.

At the end of training the callback always returns your model with the weights set to the
best weights. The callback provides a feature where it periodically queries the user to
either contine and optionally manually specify a new learning rate or halt training.
During training the calback provides useful information on the percent improvement in the
validation loss for each epoch. The is useful to decide when to halt training or manually
specifying a new learning rate.
